## Задание: Реализация полносвязной нейронной сети
### Цель
#### Реализовать простую полносвязную нейронную сеть с нуля, используя только NumPy, и применить ее для решения задачи классификации.
#### Описание задания
#### Реализуйте класс NeuralNetwork со следующей архитектурой:
Входной слой (размер зависит от данных)
Один скрытый слой (64 нейрона)
Выходной слой (размер зависит от количества классов)
Реализуйте следующие методы:
__init__: инициализация весов и смещений
forward: прямое распространение
backward: обратное распространение
train: обучение сети
predict: предсказание на новых данных
Используйте следующие функции активации:
ReLU для скрытого слоя
Softmax для выходного слоя
Используйте перекрестную энтропию в качестве функции потерь.
Протестируйте вашу сеть на наборе данных MNIST (рукописные цифры).
Требования к реализации
Используйте только NumPy для вычислений.
Реализуйте мини-пакетный градиентный спуск.
Добавьте возможность настройки гиперпараметров (скорость обучения, размер мини-пакета, количество эпох).
Дополнительные задания (по желанию)
Добавьте регуляризацию L2.
Реализуйте dropout.
Добавьте возможность использовать произвольное количество скрытых слоев.
Оценка
Корректность реализации: 50%
Качество кода и документации: 20%
Производительность на тестовом наборе MNIST: 20%
Дополнительные задания: 10%
Загрузка на GitHub
Создайте репозиторий на GitHub.
Загрузите все файлы проекта, включая код, отчет и визуализации.


In [6]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [7]:

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        # Инициализация весов и смещений
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * 0.01
        self.bias_hidden = np.zeros((1, hidden_size))

        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * 0.01
        self.bias_output = np.zeros((1, output_size))

        self.learning_rate = learning_rate

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def cross_entropy_loss(self, predictions, labels):
        n_samples = labels.shape[0]
        log_likelihood = -np.log(predictions[range(n_samples), labels])
        return np.mean(log_likelihood)

    def forward(self, X):
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = self.relu(self.hidden_input)

        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        output = self.softmax(self.output_input)

        return output

    def backward(self, X, y, output):
        n_samples = X.shape[0]
        y_one_hot = np.zeros_like(output)
        y_one_hot[np.arange(n_samples), y] = 1

        d_output_input = (output - y_one_hot) / n_samples
        d_weights_hidden_output = np.dot(self.hidden_output.T, d_output_input)
        d_bias_output = np.sum(d_output_input, axis=0, keepdims=True)

        d_hidden_output = np.dot(d_output_input, self.weights_hidden_output.T) * self.relu_derivative(self.hidden_input)
        d_weights_input_hidden = np.dot(X.T, d_hidden_output)
        d_bias_hidden = np.sum(d_hidden_output, axis=0, keepdims=True)

        self.weights_input_hidden -= self.learning_rate * d_weights_input_hidden
        self.bias_hidden -= self.learning_rate * d_bias_hidden
        self.weights_hidden_output -= self.learning_rate * d_weights_hidden_output
        self.bias_output -= self.learning_rate * d_bias_output

    def train(self, X, y, epochs=10, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                output = self.forward(X_batch)
                self.backward(X_batch, y_batch, output)

            predictions = self.predict(X)
            accuracy = np.mean(predictions == y)
            print(f'Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy:.4f}')

    def predict(self, X):
        output = self.forward(X)
        return np.argmax(output, axis=1)


In [8]:

# Загрузка и подготовка данных MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"].astype(int)

# Нормализация данных и разбиение на обучающую и тестовую выборки
X = X / 255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Параметры сети
input_size = 784  # 28x28 пикселей
hidden_size = 64
output_size = 10  # Классы от 0 до 9

# Создание и обучение модели
nn = NeuralNetwork(input_size, hidden_size, output_size, learning_rate=0.01)
nn.train(X_train, y_train, epochs=150, batch_size=32)

# Тестирование модели
y_pred = nn.predict(X_test)
test_accuracy = np.mean(y_pred == y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/150, Accuracy: 0.8471
Epoch 2/150, Accuracy: 0.8916
Epoch 3/150, Accuracy: 0.9039
Epoch 4/150, Accuracy: 0.9115
Epoch 5/150, Accuracy: 0.9170
Epoch 6/150, Accuracy: 0.9215
Epoch 7/150, Accuracy: 0.9253
Epoch 8/150, Accuracy: 0.9296
Epoch 9/150, Accuracy: 0.9339
Epoch 10/150, Accuracy: 0.9376
Epoch 11/150, Accuracy: 0.9411
Epoch 12/150, Accuracy: 0.9443
Epoch 13/150, Accuracy: 0.9471
Epoch 14/150, Accuracy: 0.9493
Epoch 15/150, Accuracy: 0.9516
Epoch 16/150, Accuracy: 0.9533
Epoch 17/150, Accuracy: 0.9549
Epoch 18/150, Accuracy: 0.9569
Epoch 19/150, Accuracy: 0.9586
Epoch 20/150, Accuracy: 0.9599
Epoch 21/150, Accuracy: 0.9614
Epoch 22/150, Accuracy: 0.9629
Epoch 23/150, Accuracy: 0.9641
Epoch 24/150, Accuracy: 0.9654
Epoch 25/150, Accuracy: 0.9663
Epoch 26/150, Accuracy: 0.9674
Epoch 27/150, Accuracy: 0.9688
Epoch 28/150, Accuracy: 0.9696
Epoch 29/150, Accuracy: 0.9703
Epoch 30/150, Accuracy: 0.9711
Epoch 31/150, Accuracy: 0.9720
Epoch 32/150, Accuracy: 0.9728
Epoch 33/150, Acc

In [9]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [10]:

class NeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.01, l2_lambda=0.0, dropout_rate=0.0):
        """
        layer_sizes: список, где первый элемент - размер входного слоя, 
                     последний элемент - размер выходного слоя, а промежуточные - размеры скрытых слоев
        learning_rate: скорость обучения
        l2_lambda: коэффициент регуляризации L2 (если 0, регуляризация отключена)
        dropout_rate: доля отключаемых нейронов для Dropout (если 0, Dropout отключен)
        """
        self.num_layers = len(layer_sizes)
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.dropout_rate = dropout_rate

        # Инициализация весов и смещений
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.01 for i in range(self.num_layers - 1)]
        self.biases = [np.zeros((1, layer_sizes[i+1])) for i in range(self.num_layers - 1)]
        
        # Маски для Dropout
        self.dropout_masks = [None] * (self.num_layers - 1)

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return (x > 0).astype(float)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def cross_entropy_loss(self, predictions, labels):
        n_samples = labels.shape[0]
        log_likelihood = -np.log(predictions[range(n_samples), labels])
        return np.mean(log_likelihood)

    def forward(self, X, training=True):
        activations = [X]
        
        for i in range(self.num_layers - 2):
            # Прямое распространение на скрытых слоях с ReLU
            z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
            a = self.relu(z)
            
            # Применение Dropout (только на этапе обучения)
            if training and self.dropout_rate > 0:
                self.dropout_masks[i] = (np.random.rand(*a.shape) > self.dropout_rate).astype(float) / (1.0 - self.dropout_rate)
                a *= self.dropout_masks[i]
                
            activations.append(a)

        # Прямое распространение на выходном слое с Softmax
        z = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        a = self.softmax(z)
        activations.append(a)

        return activations

    def backward(self, activations, y):
        n_samples = y.shape[0]
        y_one_hot = np.zeros_like(activations[-1])
        y_one_hot[np.arange(n_samples), y] = 1

        # Градиент для выходного слоя
        delta = (activations[-1] - y_one_hot) / n_samples
        
        for i in range(self.num_layers - 2, -1, -1):
            # Градиенты для весов и смещений
            d_weights = np.dot(activations[i].T, delta) + self.l2_lambda * self.weights[i]
            d_biases = np.sum(delta, axis=0, keepdims=True)

            # Обновление весов и смещений
            self.weights[i] -= self.learning_rate * d_weights
            self.biases[i] -= self.learning_rate * d_biases

            if i > 0:  # Не вычисляем дельту для входного слоя
                delta = np.dot(delta, self.weights[i].T) * self.relu_derivative(activations[i])

                # Применение Dropout во время обратного распространения
                if self.dropout_rate > 0:
                    delta *= self.dropout_masks[i - 1]

    def train(self, X, y, epochs=10, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                activations = self.forward(X_batch)
                self.backward(activations, y_batch)

            predictions = self.predict(X)
            accuracy = np.mean(predictions == y)
            print(f'Epoch {epoch + 1}/{epochs}, Accuracy: {accuracy:.4f}')

    def predict(self, X):
        activations = self.forward(X, training=False)
        return np.argmax(activations[-1], axis=1)


In [ ]:
# Загрузка и подготовка данных MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"].astype(int)

In [18]:
mnist["data"]

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:


# Нормализация данных и разбиение на обучающую и тестовую выборки
X = X / 255.0
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Параметры сети
input_size = 784  # 28x28 пикселей
hidden_layers = [64, 32]  # Произвольное количество скрытых слоев
output_size = 10  # Классы от 0 до 9
layer_sizes = [input_size] + hidden_layers + [output_size]

# Создание и обучение модели
nn = NeuralNetwork(layer_sizes, learning_rate=0.01, l2_lambda=0.01, dropout_rate=0.5)
nn.train(X_train, y_train, epochs=50, batch_size=32)

# Тестирование модели
y_pred = nn.predict(X_test)
test_accuracy = np.mean(y_pred == y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/50, Accuracy: 0.1121
Epoch 2/50, Accuracy: 0.1121
Epoch 3/50, Accuracy: 0.1121
Epoch 4/50, Accuracy: 0.2077
Epoch 5/50, Accuracy: 0.3996
Epoch 6/50, Accuracy: 0.5873
Epoch 7/50, Accuracy: 0.6747
Epoch 8/50, Accuracy: 0.7580
Epoch 9/50, Accuracy: 0.8265
Epoch 10/50, Accuracy: 0.8503
Epoch 11/50, Accuracy: 0.8632
Epoch 12/50, Accuracy: 0.8735
Epoch 13/50, Accuracy: 0.8799
Epoch 14/50, Accuracy: 0.8848
Epoch 15/50, Accuracy: 0.8885
Epoch 16/50, Accuracy: 0.8914
Epoch 17/50, Accuracy: 0.8946
Epoch 18/50, Accuracy: 0.8956
Epoch 19/50, Accuracy: 0.8970
Epoch 20/50, Accuracy: 0.8991
Epoch 21/50, Accuracy: 0.8997
Epoch 22/50, Accuracy: 0.9027
Epoch 23/50, Accuracy: 0.9022
Epoch 24/50, Accuracy: 0.9050
Epoch 25/50, Accuracy: 0.9052
Epoch 26/50, Accuracy: 0.9053
Epoch 27/50, Accuracy: 0.9061
Epoch 28/50, Accuracy: 0.9065
Epoch 29/50, Accuracy: 0.9089
Epoch 30/50, Accuracy: 0.9085
Epoch 31/50, Accuracy: 0.9086
Epoch 32/50, Accuracy: 0.9102
Epoch 33/50, Accuracy: 0.9091
Epoch 34/50, Accura

In [19]:
import pandas as pd
from pandas.api.extensions import register_dataframe_accessor

# Определим кастомный DataFrameAccessor
@register_dataframe_accessor("custom")
class CustomMethods:
    def __init__(self, df):
        self._df = df

    def square_columns(self):
        return self._df ** 2

# Теперь у любого DataFrame можно вызвать метод `custom.square_columns`
df = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
print(df.custom.square_columns())

   a   b
0  1  16
1  4  25
2  9  36


In [20]:
df.custom.square_columns()

,a,b
0,1,16
1,4,25
2,9,36
